In [1]:
import os
import requests
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup as sp

In [2]:
os.makedirs('product_links', exist_ok = True)

In [3]:
def get_soup_obj(URL, pgNo):
    
    r = requests.get(URL + f'?p={pgNo}')
    return sp(r.content, 'lxml')

In [4]:
def get_12_product_links(soup_obj):
    
    product_list = soup_obj.find('ol', {'class' : 'products list items product-items'}).find_all('li')
    return [product.find('a')['href'] for product in product_list]

In [5]:
def get_page_counts(soup_obj):
    
    no_of_items = int(soup_obj.find('div', {'class' : 'toolbar toolbar-products'}).find_all('span', {'class' : 'toolbar-number'})[-1].text)
    return int(round(no_of_items/12, 0))

In [6]:
def get_page_title(soup_obj):
    
    return soup_obj.find('span', {'data-ui-id' : 'page-title-wrapper'}).text

In [7]:
def save_as_dict(name, file):
    
    with open(f'product_links/{name}.txt', 'w') as f:
        f.write(str(file))

In [8]:
def get_product_links(URL_List, filename):
    
    links_dict = {}

    for URL in tqdm(URL_List):

        product_links = []
        soup = get_soup_obj(URL, 1)
        No_pages = get_page_counts(soup)
        page_title = get_page_title(soup)

        product_links += get_12_product_links(soup)
        sleep(2)

        for pg_No in range(2, No_pages+1):

            soup = get_soup_obj(URL, pg_No)
            product_links += get_12_product_links(soup)
            sleep(2)

        links_dict[page_title] = product_links

    save_as_dict(filename, links_dict)

In [9]:
BreakFast_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/brot-broetchen.html',
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/suesse-brotaufstriche.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/bio-brotaufstrich.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/bio-honig.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/bio-kaffee.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/bio-muesli.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/nussmus.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/pflanzendrinks.html', 
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/fruehstueckszutaten/saaten-keime.html']

In [10]:
get_product_links(BreakFast_URL, 'beakfast_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:31<00:00, 10.12s/it]


In [11]:
Superfood_URL = ['https://www.vitalia-reformhaus.de/bio-lebensmittel/superfood-lebensmittel/superfood-beeren.html',
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/superfood-lebensmittel/superfood-nuesse.html',
                 'https://www.vitalia-reformhaus.de/bio-lebensmittel/superfood-lebensmittel/superfood-pulver.html']

In [12]:
get_product_links(Superfood_URL, 'superfood_dict')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:15<00:00,  5.06s/it]
